In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [66]:
E_B = 100
P_Battery_Max = 50
init_SOC=0.5
SOC_UP =  soc_upper_bound = 0.9
SOC_FLOOR = soc_floor_bound = 0.1
eta_Bc=eta_Bd=0.95
Cost_Battery = P_Battery_Max*800+E_B*1800
Battery_lifetime = 10*365 #忽略闰年
Cost_Battery_daily = Cost_Battery/(Battery_lifetime)

# 读取数据
data_A = pd.read_csv("data/园区A.csv")
data_B = pd.read_csv("data/园区B.csv")
data_C = pd.read_csv("data/园区C.csv")

P_user_A = data_A["用电量"]
P_user_B = data_B["用电量"]
P_user_C = data_C["用电量"]

P_PV_A = data_A["光伏供电量"]
P_WG_B = data_B["风电供电量"]
P_PV_C = data_C["光伏供电量"]
P_WG_C = data_C["风电供电量"]


In [67]:
# C
def setSOC(SOC,t,value):
    SOC[t]=value
def getSOC(SOC,t):
    if t==-1:
        return SOC[-1]
    return SOC[t]
def gamma_B(P_B):
    if P_B<0:
        return eta_Bc
    else:
        return 1/eta_Bd
def get_P_B(Delta_P):#限制充放电功率
    P_Bdmax = P_Bcmax = P_Battery_Max
    if Delta_P>-P_Bdmax and Delta_P<=P_Bcmax:
        return Delta_P
    elif Delta_P>P_Bcmax:
        return P_Bcmax
    else:
        return -P_Bdmax
# 放电时为正，充电时为负
def cal_P_B(SOC,P_Battery_Max,Delta_P,t):
    P_B=get_P_B(Delta_P) 
    SOC_last = getSOC(SOC,t) 
    gamma_B_value = gamma_B(P_B)
    SOC_new = SOC_last-gamma_B_value*P_B*1/E_B
    
    # 限制充放电容量
    if SOC_new > SOC_UP:
        SOC_new = SOC_UP
    elif SOC_new < SOC_FLOOR:
        SOC_new = SOC_FLOOR
    P_B_Battery = (SOC_last-SOC_new)*E_B
    P_B_use =(SOC_last-SOC_new)*E_B/gamma_B(P_B)/1

    if(P_B_use>0):
        print(f"外部放电{abs(P_B_use)}kW,消耗电池{abs(P_B_Battery)}")
    elif(P_B_use<0):
        print(f"外部充电{abs(P_B_use)}kW,补充电池{abs(P_B_Battery)}")
    return P_B_use,P_B_Battery
def find_lack(t):
    data = P_PV_C+P_WG_C-P_user_C
    for i in range(t+1,len(data)):
        if data[i] < 0:
            return i
    return -1
def wait_sun(P_wind,Battery,SOC,t):
    j = t
    n = find_lack(t)
    lack = P_user_C[n]-P_PV_C[n]-P_WG_C[n]
    extra=0
    for k in range(j+1,n):
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,(P_PV_C[k])+P_WG_C[k]-P_user_C[k],t)
        extra+=abs(P_B_use)
    B_empty = (SOC_UP-getSOC(SOC,t))*E_B
    if(lack-extra>B_empty):
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t)
        P_wind+=P_B_use
        Battery-=P_B_Battery 
        C_wind=-P_B_use 
        return P_wind,C_wind,Battery 
    while(n<=365*24):
        j = n+1
        n = find_lack(n)
        lack = P_user_C[n]-P_PV_C[n]-P_WG_C[n]
        extra=0
        for k in range(j+1,n):
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,(P_PV_C[k])+P_WG_C[k]-P_user_C[k],t)
            extra+=abs(P_B_use)
        B_empty = (SOC_UP-getSOC(SOC,t))*E_B
        if(lack-extra>B_empty):
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t) 
            P_wind+=P_B_use
            Battery-=P_B_Battery 
            C_wind=-P_B_use 
            return P_wind,C_wind,Battery 
    return P_wind,C_wind,Battery 

def debug_Battery(Battery,SOC,t,debug):
    if debug:    
        print(f"电池电量：{Battery},{SOC[t]}")


# Battery = init_SOC*E_B or 0
def Process(SOC,Battery,P_user, P_light, P_wind,label,t,debug=False):
    if debug:
        print(f"现在是{t}:00,电池电量：{Battery}")
    C_light=0
    C_wind=0 
    C_net=0

    # 光
    if P_light > P_user:
        C_light = P_user
        P_user = 0
        P_light -= C_light
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_light,t)
        # charge
        P_light+=P_B_use 
        Battery-=P_B_Battery
        debug_Battery(Battery,SOC,t,debug)
    else:
        C_light = P_light
        P_user -= P_light
    
    if P_user == 0:
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery
    
    # 电池
    # discharge
    if(debug):
        print("自发电不足，动用电池放电")
    P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,P_user,t) #放电时为正，充电时为负
    Battery -= P_B_Battery 
    debug_Battery(Battery,SOC,t,debug)

    P_user -=P_B_use 
    if P_user==0:
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t) 
        P_wind+=P_B_use
        Battery-=P_B_Battery 
        debug_Battery(Battery,SOC,t,debug)
        C_wind=-P_B_use 
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery
    
    # 风
    if P_wind > P_user:
        C_wind = P_user
        P_user = 0
        P_wind-=C_wind

        if label == 'B':
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t)
            C_wind -= P_B_use
            P_wind +=P_B_use 
            Battery-=P_B_Battery  
            debug_Battery(Battery,SOC,t,debug)
            
        if label == 'C':
            # 额外考虑
            P_wind,C_wind,Battery = wait_sun(P_wind,Battery,SOC,t)#################
            debug_Battery(Battery,SOC,t,debug)
            
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery

        
    else:
        C_wind = P_wind
        P_user -= P_wind
    
    if P_user > 0:
        C_net = P_user
        P_user = 0
    # SOC[t]=Battery/E_B
    SOC.append(Battery/E_B)
    
    return C_light, C_wind, C_net,Battery

In [68]:
# A
Battery = init_SOC*E_B
SOC=[init_SOC]
cost = []
cost_PV_50_100 = []
cost_WG_50_100 = []
cost_net_50_100 = []
sum_A = Cost_Battery_daily
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_A[t], P_PV_A[t], 0,"A",t)
    # print(C_light, C_wind, C_net)
    cost_PV_50_100.append(C_light*0.4)
    cost_WG_50_100.append(C_wind*0.5)
    cost_net_50_100.append(C_net*1)
    cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
    sum_A+=cost[t] 
    # print(cost[t])

df = pd.DataFrame({"cost_PV":cost_PV_50_100,"cost_WG":cost_WG_50_100,"cost_net":cost_net_50_100,"cost_sum":cost})
df.to_csv("data/cost_A_50_100",index=False)

# sum_A = sum(cost_A)
# print(cost_A)
# print(sum_A)

外部放电38.0kW,消耗电池40.0
外部充电50.0kW,补充电池47.5
外部充电34.21052631578948kW,补充电池32.50000000000001
外部放电50.0kW,消耗电池52.63157894736842
外部放电26.000000000000007kW,消耗电池27.368421052631586
外部充电3.95516952522712e-15kW,补充电池4.163336342344337e-15


In [69]:
# B
Battery = init_SOC*E_B
SOC=[init_SOC]
cost = []
cost_PV_50_100 = []
cost_WG_50_100 = []
cost_net_50_100 = []
sum_B = Cost_Battery_daily
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_B[t], 0,P_WG_B[t],"B",t)
    # print(C_light, C_wind, C_net)
    cost_PV_50_100.append(C_light*0.4)
    cost_WG_50_100.append(C_wind*0.5)
    cost_net_50_100.append(C_net*1)
    cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
    # print(cost[t])
    sum_B+=cost[t]

df = pd.DataFrame({"cost_PV":cost_PV_50_100,"cost_WG":cost_WG_50_100,"cost_net":cost_net_50_100,"cost_sum":cost})
df.to_csv("data/cost_B_50_100",index=False)
# sum_B = sum(cost_B)
# print(cost_B)
# print(sum_B)

外部放电38.0kW,消耗电池40.0
外部充电27.099999999999998kW,补充电池25.744999999999997
外部放电24.457749999999997kW,消耗电池25.744999999999997
外部充电49.99999999999999kW,补充电池47.49999999999999
外部放电45.125kW,消耗电池47.5
外部充电12.925000000000006kW,补充电池12.278750000000006
外部放电11.6648125kW,消耗电池12.278749999999999
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5
外部充电34.21052631578948kW,补充电池32.50000000000001
外部放电30.875000000000007kW,消耗电池32.50000000000001
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5
外部充电34.21052631578948kW,补充电池32.50000000000001
外部放电30.875000000000007kW,消耗电池32.50000000000001
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5


In [70]:
# C
Battery = init_SOC*E_B
cost= []
cost_PV_50_100 = []
cost_WG_50_100 = []
cost_net_50_100 = []
sum_C = Cost_Battery_daily
SOC=[init_SOC]
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_C[t], P_PV_C[t],P_WG_C[t],"C",t,debug=False) 
    # print(C_light, C_wind, C_net)
    cost_PV_50_100.append(C_light*0.4)
    cost_WG_50_100.append(C_wind*0.5)
    cost_net_50_100.append(C_net*1)
    cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
    # print(cost[t])
    sum_C+=cost[t]
# print(cost_C)
# print(sum(cost_C))
# print(sum_C)
df = pd.DataFrame({"cost_PV":cost_PV_50_100,"cost_WG":cost_WG_50_100,"cost_net":cost_net_50_100,"cost_sum":cost})
df.to_csv("data/cost_C_50_100",index=False)


# bug:
# 电池只有0.1时还在放电，应该终止

外部放电38.0kW,消耗电池40.0
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5
外部充电50.0kW,补充电池47.5
外部充电34.21052631578948kW,补充电池32.50000000000001
外部放电10.739999999999899kW,消耗电池11.305263157894629
外部放电6.0199999999999845kW,消耗电池6.336842105263141
外部放电50.0kW,消耗电池52.63157894736842
外部充电11.900277008310136kW,补充电池11.305263157894629
外部放电26.000000000000007kW,消耗电池27.368421052631586
外部充电3.95516952522712e-15kW,补充电池4.163336342344337e-15


In [73]:
# 创建 DataFrame
df = pd.DataFrame({'cost_A': cost_A, 'cost_B': cost_B, 'cost_C': cost_C})

# 将 DataFrame 写入 CSV 文件
df.to_csv('data/cost_50_100.csv',index=False)

In [75]:
cost_0_0 = pd.read_csv("data/cost_0_0.csv")
cost_A_0_0 = cost_0_0["cost_A"]
cost_B_0_0 = cost_0_0["cost_B"]
cost_C_0_0 = cost_0_0["cost_C"]
print(sum(cost_A_0_0),sum(cost_B_0_0),sum(cost_C_0_0))
cost_A_50_100 = pd.read_csv("data/cost_A_50_100")
cost_B_50_100 = pd.read_csv("data/cost_B_50_100")
cost_C_50_100 = pd.read_csv("data/cost_C_50_100")
print(sum(cost_A_50_100["cost_sum"]),sum(cost_B_50_100["cost_sum"]),sum(cost_C_50_100["cost_sum"]))
print(
    sum(cost_A_50_100["cost_sum"])-sum(cost_A_0_0),
    sum(cost_B_50_100["cost_sum"])-sum(cost_B_0_0),
    sum(cost_C_50_100["cost_sum"])-sum(cost_C_0_0))



6084.875 5071.15 4963.311000000001
5970.875 5002.94924506579 4726.276138504156
-114.0 -68.20075493420973 -237.03486149584478


In [74]:
cost_0_0 = pd.read_csv("data/cost_0_0.csv")
cost_A_0_0 = cost_0_0["cost_A"]
cost_B_0_0 = cost_0_0["cost_B"]
cost_C_0_0 = cost_0_0["cost_C"]
cost_50_100 = pd.read_csv("data/cost_50_100.csv")
cost_A_50_100 = cost_50_100["cost_A"]
cost_B_50_100 = cost_50_100["cost_B"]
cost_C_50_100 = cost_50_100["cost_C"]
Delta_A_50_100 = cost_A_50_100 - cost_A_0_0
Delta_B_50_100 = cost_B_50_100 - cost_B_0_0
Delta_C_50_100 = cost_C_50_100 - cost_C_0_0
df = pd.DataFrame({"Delta_A_50_100":Delta_A_50_100,"Delta_B_50_100":Delta_B_50_100,"Delta_C_50_100":Delta_C_50_100})
df.to_csv('data/Delta_50_100.csv',index=False)
sum_A_0_0 = sum(cost_A_0_0)
sum_B_0_0 = sum(cost_B_0_0)
sum_C_0_0 = sum(cost_C_0_0)
sum_A_50_100 = sum(cost_A_50_100)
sum_B_50_100 = sum(cost_B_50_100)
sum_C_50_100 = sum(cost_C_50_100)

print(sum_A_0_0,sum_B_0_0,sum_C_0_0)
print(sum_A_50_100,sum_B_50_100,sum_C_50_100)
print(sum(Delta_A_50_100),sum(Delta_B_50_100),sum(Delta_C_50_100))

6084.875 5071.15 4963.311000000001
5970.875 5002.94924506579 6167.075000000001
-114.0 -68.2007549342105 1203.764
